In [ ]:
# import torch, os
# torch.cuda.empty_cache()
# os._exit(0)

In [1]:
!pip install -U bitsandbytes
!pip install -U transformers
!pip install -U peft
!pip install -U accelerate
!pip install -U trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 101.5 MB/s eta 0:00:0000:01:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 8.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: peft
    Found existing installation: peft 0.17.1
    Uninstalling peft-0.17.1:
      Successfully uninstalled peft-0.17.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 6.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.11.0
    Uninstalling accelerate-1.11.0:
      Successfully uninstalled accelerate-1.11.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 9.2 MB/s

In [2]:
import torch, bitsandbytes, transformers, peft, accelerate, trl

print("Torch:", torch.__version__)
print("BitsAndBytes:", bitsandbytes.__version__)
print("Transformers:", transformers.__version__)
print("PEFT:", peft.__version__)
print("Accelerate:", accelerate.__version__)
print("TRL:", trl.__version__)

2026-01-12 12:14:42.600770: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768220082.795102      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768220082.854123      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768220083.311009      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768220083.311046      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768220083.311049      55 computation_placer.cc:177] computation placer alr

Torch: 2.8.0+cu126
BitsAndBytes: 0.49.1
Transformers: 4.57.3
PEFT: 0.18.1
Accelerate: 1.12.0
TRL: 0.26.2


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer

In [21]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("hf_token")

In [22]:
!huggingface-cli login --token $secret_value_0

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `llama8B` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `llama8B`


In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer

In [25]:
import pandas as pd
import numpy as np
train_df=pd.read_csv("/kaggle/input/question-answering-training-and-testing-data/train.csv")
train_df

,instruction,responses,next_response,answer,is_human_response
0,How many seconds are in 7.8 minutes?,[],7.8 minutes is the same as 7 minutes and 0.8 m...,NaN,False
1,How many seconds are in 7.8 minutes?,['7.8 minutes is the same as 7 minutes and 0.8...,"Right, and since there are 60 seconds in a min...",NaN,False
2,How many seconds are in 7.8 minutes?,['7.8 minutes is the same as 7 minutes and 0.8...,"And since there are 60 seconds in a minute, th...",NaN,False
3,How many seconds are in 7.8 minutes?,['7.8 minutes is the same as 7 minutes and 0.8...,"So, in total, there are 420 + 48 = 468 seconds...",NaN,False
4,How many seconds are in 7.8 minutes?,['7.8 minutes is the same as 7 minutes and 0.8...,Exactly.,468,False
...,...,...,...,...,...
629465,On the Cartesian plane in which each unit is o...,['I need to find the point on the rope that is...,"To find its distance from the origin, I use th...",NaN,False
629466,On the Cartesian plane in which each unit is o...,['I need to find the point on the rope that is...,NaN,15,False
629467,Suppose we are given seven points that are equ...,[],I notice that the problem is asking about the ...,NaN,False
629468,Suppose we are given seven points that are equ...,['I notice that the problem is asking about th...,"So, if I want to find the possible values of $...",NaN,False


In [26]:
train_df.drop_duplicates(inplace=True)

In [27]:
train_df.drop(columns=["next_response","answer","is_human_response"], inplace=True)
train_df

,instruction,responses
0,How many seconds are in 7.8 minutes?,[]
1,How many seconds are in 7.8 minutes?,['7.8 minutes is the same as 7 minutes and 0.8...
2,How many seconds are in 7.8 minutes?,['7.8 minutes is the same as 7 minutes and 0.8...
3,How many seconds are in 7.8 minutes?,['7.8 minutes is the same as 7 minutes and 0.8...
4,How many seconds are in 7.8 minutes?,['7.8 minutes is the same as 7 minutes and 0.8...
...,...,...
629465,On the Cartesian plane in which each unit is o...,['I need to find the point on the rope that is...
629466,On the Cartesian plane in which each unit is o...,['I need to find the point on the rope that is...
629467,Suppose we are given seven points that are equ...,[]
629468,Suppose we are given seven points that are equ...,['I notice that the problem is asking about th...


In [28]:
train_df.isnull().sum()

instruction    0
responses      0
dtype: int64

In [29]:
train_df = train_df[train_df["responses"] != "[]"]
train_df

,instruction,responses
1,How many seconds are in 7.8 minutes?,['7.8 minutes is the same as 7 minutes and 0.8...
2,How many seconds are in 7.8 minutes?,['7.8 minutes is the same as 7 minutes and 0.8...
3,How many seconds are in 7.8 minutes?,['7.8 minutes is the same as 7 minutes and 0.8...
4,How many seconds are in 7.8 minutes?,['7.8 minutes is the same as 7 minutes and 0.8...
6,How many positive two-digit integers leave a r...,"[""So if a number leaves a remainder of 2 when ..."
...,...,...
629464,On the Cartesian plane in which each unit is o...,['I need to find the point on the rope that is...
629465,On the Cartesian plane in which each unit is o...,['I need to find the point on the rope that is...
629466,On the Cartesian plane in which each unit is o...,['I need to find the point on the rope that is...
629468,Suppose we are given seven points that are equ...,['I notice that the problem is asking about th...


In [30]:
train_df.duplicated().sum()

np.int64(1894)

In [31]:
train_df.drop_duplicates(inplace=True)
train_df

/tmp/ipykernel_55/1811325075.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop_duplicates(inplace=True)


,instruction,responses
1,How many seconds are in 7.8 minutes?,['7.8 minutes is the same as 7 minutes and 0.8...
2,How many seconds are in 7.8 minutes?,['7.8 minutes is the same as 7 minutes and 0.8...
3,How many seconds are in 7.8 minutes?,['7.8 minutes is the same as 7 minutes and 0.8...
4,How many seconds are in 7.8 minutes?,['7.8 minutes is the same as 7 minutes and 0.8...
6,How many positive two-digit integers leave a r...,"[""So if a number leaves a remainder of 2 when ..."
...,...,...
629464,On the Cartesian plane in which each unit is o...,['I need to find the point on the rope that is...
629465,On the Cartesian plane in which each unit is o...,['I need to find the point on the rope that is...
629466,On the Cartesian plane in which each unit is o...,['I need to find the point on the rope that is...
629468,Suppose we are given seven points that are equ...,['I notice that the problem is asking about th...


In [32]:
test_df=pd.read_csv("/kaggle/input/question-answering-training-and-testing-data/test.csv")
test_df.drop(columns=["next_response","answer","is_human_response"], inplace=True)

test_df = test_df[test_df["responses"] != "[]"]
test_df.drop_duplicates(inplace=True)
test_df

,instruction,responses
1,Three pencils and a jumbo eraser cost $\$1.24$...,"[""Let's call the price of a pencil p and the p..."
2,Three pencils and a jumbo eraser cost $\$1.24$...,"[""Let's call the price of a pencil p and the p..."
3,Three pencils and a jumbo eraser cost $\$1.24$...,"[""Let's call the price of a pencil p and the p..."
4,Three pencils and a jumbo eraser cost $\$1.24$...,"[""Let's call the price of a pencil p and the p..."
5,Three pencils and a jumbo eraser cost $\$1.24$...,"[""Let's call the price of a pencil p and the p..."
...,...,...
19222,Simplify $\frac{(10r^3)(4r^6)}{8r^4}$.,['To simplify a fraction involving powers of t...
19224,Find the midpoint of the line segment between ...,['I know that the midpoint of a line segment i...
19225,Find the midpoint of the line segment between ...,['I know that the midpoint of a line segment i...
19226,Find the midpoint of the line segment between ...,['I know that the midpoint of a line segment i...


In [75]:
test_df["instruction"][1]

'Three pencils and a jumbo eraser cost $\\$1.24$. Five pencils and a jumbo eraser cost $\\$1.82$. No prices include tax. In cents, what is the cost of a pencil?'

In [19]:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# secret_label = "hf_token"
# hf_token = user_secrets.get_secret(secret_label)
# print("Secret fetched successfully")
# print(hf_token)

In [20]:
# from huggingface_hub import HfApi, login

# login(token=hf_token)
# api = HfApi()
# user_info = api.whoami()
# print(user_info)

In [33]:
def format_prompt(row):
    return f"""### Instruction:
{row['instruction']}

### Response:
{row['responses']}"""

train_df["text"] = train_df.apply(format_prompt, axis=1)
test_df["text"] = test_df.apply(format_prompt, axis=1)

/tmp/ipykernel_55/1244945955.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["text"] = train_df.apply(format_prompt, axis=1)


In [34]:
train_df["text"]

1         ### Instruction:\nHow many seconds are in 7.8 ...
2         ### Instruction:\nHow many seconds are in 7.8 ...
3         ### Instruction:\nHow many seconds are in 7.8 ...
4         ### Instruction:\nHow many seconds are in 7.8 ...
6         ### Instruction:\nHow many positive two-digit ...
                                ...                        
629464    ### Instruction:\nOn the Cartesian plane in wh...
629465    ### Instruction:\nOn the Cartesian plane in wh...
629466    ### Instruction:\nOn the Cartesian plane in wh...
629468    ### Instruction:\nSuppose we are given seven p...
629469    ### Instruction:\nSuppose we are given seven p...
Name: text, Length: 396115, dtype: object

In [35]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df[["text"]])
test_dataset= test_dataset = Dataset.from_pandas(test_df[["text"]])

train_dataset

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 396115
})

In [36]:
test_dataset

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 16120
})

In [37]:
train_dataset["text"]

Column(["### Instruction:\nHow many seconds are in 7.8 minutes?\n\n### Response:\n['7.8 minutes is the same as 7 minutes and 0.8 minutes.']", "### Instruction:\nHow many seconds are in 7.8 minutes?\n\n### Response:\n['7.8 minutes is the same as 7 minutes and 0.8 minutes.'\n 'Right, and since there are 60 seconds in a minute, then there are 60 * 7 = 420 seconds in 7 minutes.']", "### Instruction:\nHow many seconds are in 7.8 minutes?\n\n### Response:\n['7.8 minutes is the same as 7 minutes and 0.8 minutes.'\n 'Right, and since there are 60 seconds in a minute, then there are 60 * 7 = 420 seconds in 7 minutes.'\n 'And since there are 60 seconds in a minute, then there are 60 * 0.8 = 48 seconds in 0.8 minutes.']", "### Instruction:\nHow many seconds are in 7.8 minutes?\n\n### Response:\n['7.8 minutes is the same as 7 minutes and 0.8 minutes.'\n 'Right, and since there are 60 seconds in a minute, then there are 60 * 7 = 420 seconds in 7 minutes.'\n 'And since there are 60 seconds in a minu

In [38]:
train_dataset = train_dataset.remove_columns(["__index_level_0__"])
test_dataset = test_dataset.remove_columns(["__index_level_0__"])

In [39]:
train_dataset

Dataset({
    features: ['text'],
    num_rows: 396115
})

In [40]:
test_dataset

Dataset({
    features: ['text'],
    num_rows: 16120
})

In [41]:
test_dataset["text"]

Column(['### Instruction:\nThree pencils and a jumbo eraser cost $\\$1.24$. Five pencils and a jumbo eraser cost $\\$1.82$. No prices include tax. In cents, what is the cost of a pencil?\n\n### Response:\n["Let\'s call the price of a pencil p and the price of a jumbo eraser e. Then we can write two equations."]', '### Instruction:\nThree pencils and a jumbo eraser cost $\\$1.24$. Five pencils and a jumbo eraser cost $\\$1.82$. No prices include tax. In cents, what is the cost of a pencil?\n\n### Response:\n["Let\'s call the price of a pencil p and the price of a jumbo eraser e. Then we can write two equations."\n \'We have $3p+e=1.24$ and $5p+e=1.82$.\']', '### Instruction:\nThree pencils and a jumbo eraser cost $\\$1.24$. Five pencils and a jumbo eraser cost $\\$1.82$. No prices include tax. In cents, what is the cost of a pencil?\n\n### Response:\n["Let\'s call the price of a pencil p and the price of a jumbo eraser e. Then we can write two equations."\n \'We have $3p+e=1.24$ and $5p

In [42]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
import torch

In [45]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import torch

base_model = "mistralai/Mistral-7B-Instruct-v0.3"

bnb_config = BitsAndBytesConfig(  
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    # bnb_4bit_compute_dtype=torch.float16, 
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    # bnb_4bit_compute_dtype=torch.float16, 
    device_map="auto",
    trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

(True, True)

In [49]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [60]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps= 25,
    logging_steps= 25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="none",
    # eval_strategy="steps",  
    # eval_steps= 25
)

In [61]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    # eval_dataset=test_dataset,
    peft_config=peft_config,
    args=training_arguments,
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/396115 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/396115 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/396115 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,0.703800
50,0.884100
75,0.640000
100,0.840000
125,0.594600
150,0.776800
175,0.609400
200,0.769100
225,0.588400
250,0.756300


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/

KeyboardInterrupt: 

In [63]:
model.eval()


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): MistralForCausalLM(
          (model): MistralModel(
            (embed_tokens): Embedding(32768, 4096)
            (layers): ModuleList(
              (0-31): 32 x MistralDecoderLayer(
                (self_attn): MistralAttention(
                  (q_proj): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=4096, out_features=64, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=64, out_features=4096, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
     

In [76]:
test_df["instruction"][1]

'Three pencils and a jumbo eraser cost $\\$1.24$. Five pencils and a jumbo eraser cost $\\$1.82$. No prices include tax. In cents, what is the cost of a pencil?'

In [69]:
prompt = "Three pencils and a jumbo eraser cost $\\$1.24$. Five pencils and a jumbo eraser cost $\\$1.82$. No prices include tax. In cents, what is the cost of a pencil?"

inputs = tokenizer(
    prompt,
    return_tensors="pt",
    padding=False,
    truncation=True,
).to(model.device)

In [70]:
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id,
    )

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [71]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Three pencils and a jumbo eraser cost $\$1.24$. Five pencils and a jumbo eraser cost $\$1.82$. No prices include tax. In cents, what is the cost of a pencil?### I'm having trouble understanding how to use the given information to find the price of a pencil in cents.

I can see that three pencils and a jumbo eraser cost $1.24$, so I could divide by 3 to get the cost per pencil. However, that would give me an answer in dollars, not cents. To convert to cents, I need to multiply by 100. But then I have a fraction, which I don't want.

So maybe I should try another approach: I can subtract the cost of the jumbo eraser from each total cost, since it's the same in both cases. Then I can divide by 2 to get the cost per pencil. This would also give me an answer in dollars, but I can convert to cents by multiplying by 100 again.

What do you think? Which approach is better or more accurate


In [91]:
prompt= "Three pencils and a jumbo eraser cost $\\$1.24$. Five pencils and a jumbo eraser cost $\\$1.82$. No prices include tax. In cents, what is the cost of a pencil?"
inputs = tokenizer(
    prompt,
    return_tensors="pt",
    padding=False,
    truncation=True,
).to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.2,
        top_p=0.8,
        repetition_penalty=1.2,
        eos_token_id=tokenizer.eos_token_id,
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Three pencils and a jumbo eraser cost $\$1.24$. Five pencils and a jumbo eraser cost $\$1.82$. No prices include tax. In cents, what is the cost of a pencil? ### I need to find out how much one pencil costs in cents.
Let's call that x. Then we can set up two equations based on the given information:

x + 3(x) = 124 for three pencils and an eraser
5x + 3(x) = 182 for five pencils and an eraser

To solve these equations, we can subtract the first equation from the second one: (5x - x) + 0 = 68. This simplifies to 4x = 68. Dividing both sides by 4 gives us x = 17. So each pencil costs $1.70 in dollars or 170 cents.


In [86]:
prompt= """Steve says to Jon, "I am thinking of a polynomial whose roots are all positive integers. The polynomial has the form $P(x) = 2x^3-2ax^2+(a^2-81)x-c$ for some positive integers $a$ and $c$. Can you tell me the values of $a$ and $c$?"\n\nAfter some calculations, Jon says, "There is more than one such polynomial."\n\nSteve says, "You\'re right. Here is the value of $a$." He writes down a positive integer and asks, "Can you tell me the value of $c$?"\n\nJon says, "There are still two possible values of $c$."\n\nFind the sum of the two possible values of $c$."""

inputs = tokenizer(
    prompt,
    return_tensors="pt",
    padding=False,
    truncation=True,
).to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.3,
        top_p=0.7,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id,
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Steve says to Jon, "I am thinking of a polynomial whose roots are all positive integers. The polynomial has the form $P(x) = 2x^3-2ax^2+(a^2-81)x-c$ for some positive integers $a$ and $c$. Can you tell me the values of $a$ and $c$?"

After some calculations, Jon says, "There is more than one such polynomial."

Steve says, "You're right. Here is the value of $a$." He writes down a positive integer and asks, "Can you tell me the value of $c$?"

Jon says, "There are still two possible values of $c$."

Find the sum of the two possible values of $c$. How many positive integers $a$ satisfy the given condition?

To find the number of positive integers $a$ that make the polynomial have all positive integer roots, I need to use the rational root theorem, which says that any rational root of the polynomial must be of the form $\\frac{p}{q}$, where $p$ is a factor of the constant term and $q$ is a factor of the leading coefficient. Since the polynomial has no rational roots, it means that $p$ mus

In [95]:
prompt = "How many positive two-digit integers leave a remainder of 2 when divided by 8?"

inputs = tokenizer(
    prompt,
    return_tensors="pt",
    padding=False,
    truncation=True,
).to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.3,
        top_p=0.8,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id,
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


How many positive two-digit integers leave a remainder of 2 when divided by 8? ### I notice that the first two digits of a number must be even, since 2 is the remainder.
What are some examples of such numbers?

I also know that the second digit cannot be 0, because then the number would not have a remainder of 2 when divided by 8.
So what are some other possibilities for the second digit?

I think it could be 2, 4, or 6, but I'm not sure about the others.
How can I check if they work?

To check if a number has a remainder of 2 when divided by 8, I need to subtract 8 from it and see if the result is even.
If it is, then the original number had a remainder of 2.

For example, if I have 12, which is even, then 12 - 8 = 4, which is also even, so 12 has a remainder of 2 when divided by 8.
Similarly, if I have 32, which is even, then 32 - 8 = 24, which is even, so 32 has a remainder of 2 when divided by
